In [1]:
import tensorflow as tf
from utils import cifar10_load
from models import build_ResNet
from tensorflow.keras.models import Model


(x_train, y_train), (x_dev, y_dev), (x_test, y_test) = cifar10_load()

# 这里用name_scope是无效的
model = tf.keras.models.load_model('ResNet32_acc893.h5')
base_varlist = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [2]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

lr_init 0.1, 40epochs后调为0.01, 60epochs后调为0.001, 共100epochs

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Sequential

# 这里name_scope是有效的
detector = Sequential([
    Conv2D(96, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal', input_shape=(32,32,16)),
    MaxPooling2D(),
    Conv2D(192, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal'),
    MaxPooling2D(),
    Conv2D(192, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal'),
    Conv2D(2, (1,1), strides=(1,1), padding='same', activation='relu', kernel_initializer='he_normal'),
    GlobalAveragePooling2D()])
detect_pipeline = Model(inputs=model.input, outputs=Activation('softmax', name="detect_softmax")(detector(model.get_layer('activation_10').output)))

In [4]:
from cleverhans.utils_keras import KerasModelWrapper
from modified_PGD import ProjectedGradientDescent
import numpy as np

x = tf.placeholder(tf.float32, shape=(None,32,32,3))
x_label = tf.placeholder(tf.float32, shape=(None,2))
adv_label = tf.placeholder(tf.float32, shape=(None,2))

model_wrap = KerasModelWrapper(model)
detect_wrap = KerasModelWrapper(detect_pipeline)
pgd_wrap = ProjectedGradientDescent(model_wrap, detect_wrap)

pgd_params = {'eps':0.3, 'eps_iter':0.25, 'nb_iter':10, 'y':adv_label, 'ord':np.inf,
              'clip_min':0, 'clip_max':1, 'y_target':None, 'rand_init':None, 'rand_minmax':0.3, 'sanity_checks':True}

adv_x = pgd_wrap.generate(x, **pgd_params)

/tf/cleverhans-master/cleverhans/attacks_tf.py:27: UserWarning: attacks_tf is deprecated and will be removed on 2019-07-18 or after. Code should import functions from their new locations directly.
  warnings.warn("attacks_tf is deprecated and will be removed on 2019-07-18"


Instructions for updating:
Use tf.cast instead.


/tf/cleverhans-master/cleverhans/compat.py:23: UserWarning: <function reduce_max_v1 at 0x7f5784cdaea0> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "


Instructions for updating:
dim is deprecated, use axis instead


/tf/cleverhans-master/cleverhans/compat.py:23: UserWarning: <function reduce_sum_v1 at 0x7f5784cda1e0> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "


In [5]:
def generate_labels(length):
    adv_labels = np.zeros([length, 2])
    normal_labels = np.copy(adv_labels)
    adv_labels[:, 1] = 1 
    normal_labels[:, 0] = 1
    
    return adv_labels, normal_labels

In [6]:
def initialize_uninitialized(sess):
       global_vars = tf.global_variables()
       is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
       not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]
       #print([str(i.name) for i in not_initialized_vars]) # only for testing
       if len(not_initialized_vars):
            sess.run(tf.variables_initializer(not_initialized_vars))

In [7]:
total_varlist = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
new_varlist = list(set(total_varlist) - set(base_varlist))

In [8]:
batch_size = 32
training_epochs = 100

detect_x = detect_pipeline(x)
detect_adv = detect_pipeline(adv_x)

sess = tf.keras.backend.get_session()

loss1 = tf.keras.backend.categorical_crossentropy(x_label, detect_x)
loss2 = tf.keras.backend.categorical_crossentropy(adv_label, detect_adv)
total_loss = tf.reduce_sum(loss1 + loss2)
train_step = tf.train.AdamOptimizer(1e-4).minimize(total_loss, var_list=new_varlist)

correct_prediction1 = tf.equal(tf.argmax(detect_x, 1), tf.argmax(x_label, 1))
correct_prediction2 = tf.equal(tf.argmax(detect_adv, 1), tf.argmax(adv_label, 1))
accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, "float"), name='accuracy1')
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, "float"), name='accuracy2')
accuracy = 0.5*accuracy1 + 0.5*accuracy2

initialize_uninitialized(sess)

In [9]:
tf.summary.scalar('total_loss',total_loss)
tf.summary.scalar('accuracy_clean', accuracy1)
tf.summary.scalar('accuracy_adv', accuracy2)
writer = tf.summary.FileWriter('/tflog',sess.graph)
merged = tf.summary.merge_all()

In [10]:
for i in range(training_epochs):
    adv_labels, normal_labels = generate_labels(batch_size)
    for j in range(len(x_train)//batch_size-1):
        _, tf_log = sess.run([train_step, merged], feed_dict={x: x_train[j*batch_size:(j+1)*batch_size],
                                               x_label: normal_labels, adv_label: adv_labels})
        writer.add_summary(tf_log,j+i*(len(x_train)//batch_size-1))
    adv_labels, normal_labels = generate_labels(len(x_dev[:500]))
    print("iteration: %d, test accuracy: %g" % (i, accuracy.eval(session=sess, feed_dict={x: x_dev[:500], 
                                                                                          x_label: normal_labels, 
                                                                                          adv_label: adv_labels})))

iteration: 0, test accuracy: 0.988
iteration: 1, test accuracy: 0.998


KeyboardInterrupt: 

In [ ]:
detect_pipeline.save('detector.h5')

In [ ]:
outlayers = [layer.get_output_at(-1) for layer in detect_pipeline.layers]
test = Model(inputs=detect_pipeline.input, outputs=outlayers)

将Dectector和原来model的隐藏层连起来

In [ ]:
detect_pipeline.summary()

In [ ]:
from cleverhans.utils_keras import KerasModelWrapper

x = tf.placeholder(tf.float32, shape=(None,32,32,3))
y = tf.placeholder(tf.float32, shape=(None,10))
model_wrap = KerasModelWrapper(model)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Sequential

hidden = model.get_layer('activation_10').output

conv1 = Conv2D(96, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='uniform',
              name='detector_conv1')(hidden)
mp1 = MaxPooling2D(name='detector_mp1')(conv1)
conv2 = Conv2D(192, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='uniform',
              name='detector_conv2')(mp1)
mp2 = MaxPooling2D(name='detector_mp2')(conv2)
conv3 = Conv2D(192, (3,3), strides=(1,1), padding='same', activation='relu', kernel_initializer='uniform',
              name='detector_conv3')(mp2)
conv4 = Conv2D(2, (1,1), strides=(1,1), padding='same', activation='relu', kernel_initializer='uniform',
              name='detector_conv4')(conv3)
GAP = GlobalAveragePooling2D(name='detector_GAP')(conv4)

detector_out = Activation('softmax', name='detector_softmax')(GAP)
detect_pipeline = Model(inputs=model.input, outputs=detector_out)

In [ ]:
for layer in model.layers:
    layer.trainable = False

detect_pipeline = Model(inputs=model.input, outputs=Activation('softmax', name="detect_softmax")(detector(model.get_layer('add_3').output)))
detect_wrap = KerasModelWrapper(detect_pipeline)

In [ ]:
detect_pipeline.summary()

In [ ]:
from utils import cifar10_load
(x_train, y_train), (x_dev, y_dev), (x_test, y_test) = cifar10_load()
import numpy as np
detect_pipeline.predict(x_dev[0].reshape(-1,32,32,3))

In [ ]:
detect_pipeline.save('pipeline_test.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
detect_pipeline = tf.keras.models.load_model('pipeline_test.h5')
outlayers = [layer.output for layer in detect_pipeline.layers]

In [ ]:
outlayers[31]

In [ ]:
test = Model(inputs=detect_pipeline.input, outputs=outlayers[32])

检测器对对抗样本输出为1,正常样本输出为0

In [ ]:
from modified_PGD import ProjectedGradientDescent
import numpy as np

pgd_wrap = ProjectedGradientDescent(model_wrap, detect_wrap)
pgd_params = {'eps':0.3, 'eps_iter':0.25, 'nb_iter':10, 'y':np.array([[0, 1]]), 'ord':np.inf,
              'clip_min':0, 'clip_max':1, 'y_target':None, 'rand_init':None, 'rand_minmax':0.3, 'sanity_checks':True}
adv_x = pgd_wrap.generate(x, **pgd_params)

#train_pipeline = Model(inputs=x, outputs=[model(x), model(adv_x), detect_pipeline(x), detect_pipline(adv_x)])

train pipeline没有办法抽象成Model储存，只能手写训练流程

In [ ]:
model.evaluate(x_dev, y_dev)

In [ ]:
hidden = model_wrap.get_layer(x, 'activation_10')
detector_out = detector(hidden)

In [ ]:
model.summary()

In [ ]:
detector.summary()

In [ ]:
detect_pipeline.summary()

In [ ]:
with tf.variable_scope('base_model'):
    model = build_ResNet(depth=32)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(x_train)

In [ ]:
lr_adjust = [0.1, 0.1, 0.01, 0.001, 0.001]
for lr in lr_adjust:
    model.compile(
        optimizer=tf.keras.optimizers.SGD(lr=lr, momentum=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), steps_per_epoch=len(x_train)//32,
                        validation_data=(x_dev, y_dev), epochs=20)

In [ ]:
model.save('ResNet32_acc893.h5')